In [1]:
from hart_tools import (
    prep_manifest,
    read_hart_cvr, 
    read_cvrs, 
    check_for_contest,
    filter_cvr_contest,
    tabulate_styles
)

from IPython.core.interactiveshell import InteractiveShell

from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean, check_audit_parameters, find_margins,\
    find_p_values, find_sample_size, new_sample_size, summarize_status,\
    write_audit_parameters, sort_cvr_sample_num, consistent_sampling

import os
import io
import re
import numpy as np
import math
import csv
import pandas as pd
import warnings
import copy
import xml.etree.ElementTree as ET
import xml.dom.minidom
import cryptorandom
from cryptorandom.cryptorandom import SHA256, int_from_hash_py3, int_from_hash

In [53]:
# read in CVRs
cvr_list = read_cvrs(cvr_folder = "Data/hart/OC2021/oc_cvrs_for_testing_v2")
# assign random sample nums to all CVRs
CVR.assign_sample_nums(cvr_list, prng=SHA256(32))
# read in manifest
manifest = pd.read_csv("Data/hart/OC2021/oc_manifest_sample.csv")

In [3]:
# set values
seed = 12345678901234567890  # use, e.g., 20 rolls of a 10-sided die. Seed doesn't have to be numeric
replacement = False

risk_function = "alpha_mart"
risk_fn = lambda x: TestNonnegMean.alpha_mart(x, N=max_cards)
g = 0.1
max_cards = 9 + 1 # number in the sample plus 1
error_rate = 0.002

In [4]:
# contests to audit. Edit with details of your contest (eg., Contest 1 is the pres race)
contests = {'PRESIDENT AND VICE PRESIDENT':{'risk_limit':0.05,
                     'cards': 9,
                     'choice_function':'plurality',
                     'n_winners':1,
                     'candidates':['JOSEPH R. BIDEN\nKAMALA D. HARRIS',
                                   'DONALD J. TRUMP\nMICHAEL R. PENCE'],
                     'reported_winners' : ['DONALD J. TRUMP\nMICHAEL R. PENCE'],
                    }
           }

In [5]:
all_assertions = Assertion.make_all_assertions(contests)

In [6]:
all_assertions

{'PRESIDENT AND VICE PRESIDENT': {'DONALD J. TRUMP\nMICHAEL R. PENCE v JOSEPH R. BIDEN\nKAMALA D. HARRIS': <assertion_audit_utils.Assertion at 0x1252b2a30>}}

In [7]:
cvr_list, phantom_vrs = CVR.make_phantoms(max_cards, cvr_list, contests, use_style=True, prefix='phantom-1-')
print(f"Created {phantom_vrs} phantom records")

Created 4 phantom records


In [8]:
cvr_list[9].phantom

True

In [9]:
min_margin = find_margins(contests, all_assertions, cvr_list, use_style=False)

In [10]:
min_margin

0.07692307692307687

In [11]:
check_audit_parameters(risk_function, g, error_rate, contests)

In [12]:
# find initial sample size
rf = lambda x: risk_fn(x)[1]   # p_history is the second returned value
ss_fn = lambda m, r: TestNonnegMean.initial_sample_size(\
                        risk_function=rf, N=max_cards, margin=m, polling=False, \
                        error_rate=error_rate, alpha=r, reps=10) # change for comparison audits
sample_size = find_sample_size(contests, all_assertions, sample_size_function=ss_fn)  
print(sample_size)

1.0


In [45]:
consistent_sampling(cvr_list, {'PRESIDENT AND VICE PRESIDENT' : 1})

['85_1']

In [47]:
# set mvr_list to be the same as cvr_list for now -- sample order??
mvr_list = copy.deepcopy(cvr_list)

In [59]:
manifest.head()

,Container,Tabulator,Batch Name,Number of Ballots
0,Mail,1,1,60
1,Mail,1,2,21
2,Mail,1,3,123
3,Mail,1,4,59
4,Mail,1,5,87
